# Best Practices for Prompt Engineering



In [2]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version="2024-02-01"
    )

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"

# 1. Use the latest model

Use the latest model for best results.

# 2. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [4]:
# Define las instrucciones que guiarán al modelo para que resuma el texto dado, como una lista de viñetas.
# Luego, utiliza esas instrucciones para construir un "prompt" (entrada) que incluye el contexto del texto a resumir.

instrucctiones = 'Resumeme el siguiente texto como si fuera una lista de viñetas'

prompt = f"{instrucctiones} ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###"
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt}],
        max_tokens=400,)

print(response.choices[0].message.content)

- OpenAI y Microsoft amplían su asociación.
- Inversión multimillonaria de Microsoft a largo plazo.
- Continúan los inversiones previas de Microsoft en 2019 y 2021.
- Permite a OpenAI seguir investigando y desarrollando IA de manera segura y útil.
- OpenAI es una empresa con beneficios limitados y está gobernada por una organización sin fines de lucro.
- Estructura que permite recaudar capital sin comprometer la misión de compartir beneficios y priorizar la seguridad.
- Microsoft comparte la visión y los valores de OpenAI.
- La asociación es clave para el progreso de OpenAI.


# 3. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

In [5]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Escribeme un poema sobre futbol',}],
        max_tokens=40,)

print(response.choices[0].message.content)


En el verde campo, el sol despierta,  
el silbato suena, la emoción se alerta,  
veinte corazones laten como uno,  
el deporte rey, pasión que se


In [6]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Escribeme un poema sobre tenis, \
                centrate es describir la pasion de los aficionados',}],
        max_tokens=400,)

print(response.choices[0].message.content)



En la cancha el rugir resuena,  
el sol brilla, la esperanza plena,  
dos raquetas, un instante eterno,  
la pasión del juego, fuego interno.  

Bajo el cielo, la línea traza,  
donde el sudor y el amor se abrazan,  
gritos y aplausos, el aire vibrante,  
cada punto es danza, un lujo constante.  

Los aficionados, corazones latiendo,  
con cada saque, su ánimo enciendo,  
los colores ondean, banderas ondean,  
la magia del tenis, juntos pasean.  

En los asientos, un mundo de sueños,  
suspiros y saltos, momentos pequeños,  
las miradas fijas, los rostros encendidos,  
terrenos de gloria, pasiones perdidos.  

El toque sutil de la bola en acción,  
el eco del juego, pura adicción,  
en cada set, en cada desafío,  
la comunidad vive, un amor tardío.  

Victores y lágrimas, un manto de emoción,  
cada partido, una nueva lección,  
la vida se juega línea a línea,  
en el tenis hallamos nuestra doctrina.  

Así avanza la pasión desbordante,  
como un río que fluye, siempre constante,  
los

# 4. Articulate the desired output format through examples (example 1, example 2). 

In [8]:
#Escribe las instrucciones aquí. Las instrucciones deben especificar que el modelo debe:
# 1. Extraer los nombres de las empresas y los años del texto.
# 2. Generar la salida como un diccionario con 'text', 'start' y 'end'.
instrucctiones = ' Extraeme los nombres de las empresas y los años del texto \
                    Generate output as {{\"text\": \"OpenAI\", \"start\": 28, \"end\": 34}} ' 

prompt = f"{instrucctiones}  \
        ### We’re happy to announce that OpenAI and Microsoft are extending our partnership. This multi-year, \
        multi-billion dollar investment from Microsoft follows their previous investments in 2019 and 2021, \
        and will allow us to continue our independent research and develop AI that is increasingly safe, useful, and powerful. \n\n ### "


response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



Aquí tienes el formato solicitado para las empresas y los años extraídos del texto:

```json
[
    {"text": "OpenAI", "start": 28, "end": 34},
    {"text": "Microsoft", "start": 39, "end": 48},
    {"text": "2019", "start": 97, "end": 101},
    {"text": "2021", "start": 106, "end": 110}
]
```


In [9]:

# Las instrucciones deben pedir al modelo que extraiga los siguientes tipos de entidades del texto:
# 1. Nombres de empresas.
# 2. Años.
# 3. Temas específicos relacionados con el contenido.
# 4. Temas generales y abarcativos.
# El formato de salida esperado debe incluir listas separadas por comas para cada tipo de entidad.

instrucctiones = "Extraeme los nombres de las empresas, los años, los temas específicos y los temas generales del texto.\
                    Genera la salida como una lista separada por comas para cada tipo de entidad "


prompt = f"{instrucctiones}  \
        '\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                '"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)



Empresas: OpenAI, Microsoft  
Años: 2019, 2021  
Temas específicos: inversión, asociación, investigación independiente, desarrollo de IA  
Temas generales: tecnología, inteligencia artificial, colaboración empresarial  


# 5. Start with zero-shot, then few-shot (example), neither of them worked, then fine-tune (To update)

In [10]:
#THIS IS zero-shot

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, investment, multi-year, multi-billion dollar, AI, independent research, safe, useful, powerful.


In [13]:
#Few Shot learning


# Escribir las instrucciones para que el modelo extraiga palabras clave del texto, y formatee la salida como listas separadas por comas.

instrucctiones = "Extraeme palabras clave de los textos correspondientes. Genera la salida como listas separadas por comas."

prompt = f"{instrucctiones} \
                Text: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ###\n\
                Text: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords: language models, text processing, API.\n\n\
                ##W"

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":prompt},
                {"role":"user","content": '\n\
                Text: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords:',}],
        max_tokens=400,)

print(response.choices[0].message.content)



OpenAI, Microsoft, partnership, multi-year investment, multi-billion dollar, independent research, AI, safe, useful, powerful.


# 6. Reduce “fluffy” and imprecise descriptions

In [14]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the NextGen Car Seat, where safety meets innovation. Designed with advanced impact protection technology and ergonomic support, this car seat adapts as your child grows, ensuring maximum comfort and security from infancy to toddlerhood. Its sleek, modern design and easy-install features make every journey a breeze, while breathable materials keep your little one cool and cozy. Drive with peace of mind knowing your child is protected in style!


In [15]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response.choices[0].message.content)



Introducing the next generation of car seats, the SafeJourney Pro – a revolutionary blend of safety, comfort, and technology. Crafted with advanced impact-absorbing materials, this car seat ensures maximum protection for your child during every journey. Featuring an intuitive installation system and adjustable settings, the SafeJourney Pro grows with your child, accommodating them from infancy to toddlerhood. Enhanced with smart technology, it provides real-time monitoring and alerts, ensuring peace of mind for parents on the go. Experience the perfect harmony of design and innovation with the SafeJourney Pro, where every ride is a safe adventure.


# 7. Instead of just saying what not to do, say what to do instead

In [16]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. Can you describe what happens when you try to log in?


In [17]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in to your account. There can be several reasons for login issues, such as incorrect passwords or account settings. I recommend checking our help article at www.samplewebsite.com/help/faq for troubleshooting tips that might help you resolve this issue. If you need further assistance after reviewing the article, please let me know!


# 8. Code Generation Specific - Use “leading words” to nudge the model toward a particular pattern

In [19]:
# Escribir las instrucciones para crear una función simple en Python que:
# 1. Pida un número en millas.
# 2. Convierta las millas a kilómetros.

prompt = "Dame un número en millas y conviertemelas a kilometros \
          dame la salida solo de los kilometros y abajo de las millas "

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":prompt,}],
        max_tokens=400,)

print(response.choices[0].message.content)


Por supuesto. Elige un número en millas y lo convertiré a kilómetros. Por ejemplo, si me das el número 10, el resultado será:

16.09 kilómetros  
10 millas

Por favor, indícame el número en millas que deseas convertir.


## Ejercicio 1: Estructuración de Prompts con Separadores

Instrucciones: Escribe un prompt que solicite a la IA resumir el siguiente texto en una lista de puntos clave. Asegúrate de usar separadores claros (### o """).

Texto:
OpenAI y Microsoft han ampliado su colaboración. Microsoft está invirtiendo miles de millones de dólares durante varios años. Esta inversión permitirá que OpenAI continúe investigando de forma independiente y desarrolle IA segura y útil. OpenAI sigue siendo una empresa de beneficios limitados, regida por su organización sin fines de lucro.


In [22]:

response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": "Reume el texto en una lista de puntos claves ###"}],
    max_tokens=400,
)

print(response.choices[0].message.content)



Por supuesto, pero necesitaré que me proporciones el texto que deseas que resuma. ¿Puedes compartirlo, por favor?


## Ejercicio 2: Descripción Detallada del Formato Deseado

Instrucciones: Escribe un prompt para que la IA extraiga nombres de empresas y años del siguiente texto. Indica explícitamente que se desea el resultado en formato JSON con índices de inicio y fin.

Texto:
OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA.

Formato de salida deseado:

{"text": "OpenAI", "start": 0, "end": 6}
{"text": "Microsoft", "start": 10, "end": 19}
{"text": "2019", "start": 35, "end": 39}
{"text": "2021", "start": 50, "end": 54}

In [23]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity extraction assistant."},
                {"role":"user","content": "Extraeme los nombres de la mpresa y los años del texto ###\
                 OpenAI y Microsoft han colaborado desde 2019, con inversiones adicionales en 2021 para desarrollar nuevas soluciones de IA"}],
    max_tokens=400,
)

print(response.choices[0].message.content)


Nombres de la empresa: OpenAI, Microsoft  
Años: 2019, 2021


## Ejercicio 3: Generación de Texto Inspirador

Instrucciones: Redacta un prompt que solicite a la IA escribir un poema inspirador. Escribe un poema corto sobre la colaboración entre OpenAI y Microsoft.


In [24]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a creative assistant."},
                {"role":"user","content": "escribeme un poema corto sobre la colaboracion entre OpenAi  y Microsoft"}],
    max_tokens=400,
)

print(response.choices[0].message.content)

En la danza de bits y bytes,  
dos mentes se unen en la creación,  
OpenAI y Microsoft, un faro brillante,  
tejiendo sueños de innovación.  

Inteligencia que vuela,  
conectando corazones en red,  
la tecnología abraza al hombre,  
donde el futuro ya es un deber.  

Codo a codo, ¡qué sinfonía!  
El código se vuelve poesía,  
en la colaboración que florece,  
un mundo nuevo se establece.  

Alma de máquina y humanidad,  
juntos forjando claridad,  
en este viaje de exploración,  
OpenAI y Microsoft, unión de visión.


## Ejercicio 4: Extracción de Entidades Clave

Instrucciones: Escribe un prompt que pida a la IA extraer nombres de empresas, años, temas específicos y temas generales del siguiente texto:

Texto:
OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021

Formato esperado:

Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: colaboración  
General themes: asociación, tecnología, inteligencia artificial 



In [29]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are an entity and theme extraction assistant."},
                {"role":"user","content": "Extraeme los nombres de las empresas, los años, los temas específicos y los temas generales del texto.\
OpenAI y Microsoft han trabajado juntos desde 2019 y reforzaron su colaboración en 2021"}],
    max_tokens=400,
)

print(response.choices[0].message.content)


**Nombres de las empresas:**
- OpenAI
- Microsoft

**Años:**
- 2019
- 2021

**Temas específicos:**
- Colaboración entre OpenAI y Microsoft

**Temas generales:**
- Alianzas empresariales
- Innovación tecnológica


## Ejercicio 5: Descripciones Concisas de Productos

Instrucciones: Escribe un prompt para que la IA describa un nuevo tipo de mochila tecnológica en menos de 4 líneas. La mochila tiene cargador solar, materiales resistentes al agua y un diseño ergonómico.



In [31]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a marketing assistant."},
                {"role":"user","content": "describe un nuevo tipo de mochila que tiene cargador solar,materiales resistentes al agua en menos de 4 lineas" }],
    max_tokens=400,
)

print(response.choices[0].message.content)


Presentamos la mochila EcoCharge: diseñada con un cargador solar integrado, perfecto para mantener tus dispositivos siempre cargados. Sus materiales resistentes al agua garantizan la protección de tus pertenencias en cualquier clima, combinando funcionalidad y estilo. Ideal para aventureros y viajeros urbanos.
